In [ ]:
%load_ext dotenv

# Use find_dotenv to locate the file
%dotenv

import os

In [ ]:
import pandas as pd
import json
import requests
import datetime

In [8]:
transcripts_folder = "company_transcripts/"

In [2]:
from os import listdir
from os.path import isfile, join
transcripts_files = [f for f in listdir(transcripts_folder) if isfile(join(transcripts_folder, f)) and f.split(".")[-1] == 'json']
transcripts_files[0:5]

['DLTR.json', 'IDXX.json', 'VTR.json', 'KEY.json', 'WLTW.json']

In [ ]:
symbol = "MSFT"

In [ ]:
#res = requests.get(f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={os.getenv("ALPHAVANTAGE_API")}')
res = requests.get(f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey={os.getenv("ALPHAVANTAGE_API")}')
res.content

In [ ]:
dict_data = json.loads(res.content)
dict_data

In [ ]:
dict_data.keys()

In [ ]:
for tmp_field in dict_data.keys():
    if 'Meta' in tmp_field:
        hdr_metadata = tmp_field
    elif 'Time' in tmp_field and 'Serie' in tmp_field:
        hdr_ts = tmp_field

# retrieve symbol from data

In [ ]:
dict_data[hdr_metadata].keys()

In [ ]:
for tmp_field in dict_data[hdr_metadata].keys():
    if 'Symbol' in tmp_field:
        hdr_symbol = tmp_field
        break

In [ ]:
tmp_symbol = dict_data[hdr_metadata][hdr_symbol]
tmp_symbol

# timeseries

In [ ]:
len(dict_data[hdr_ts])

In [ ]:
dict_data[hdr_ts].keys()

In [ ]:
for date_tmp in dict_data[hdr_ts]:
    data_dim = dict_data[hdr_ts][date_tmp].keys()
    break
data_dim

In [ ]:
for tmp_key in data_dim:
    if 'open' in tmp_key:
        hdr_open = tmp_key
    elif 'high' in tmp_key:
        hdr_high = tmp_key
    elif 'low' in tmp_key:
        hdr_low = tmp_key
    elif 'close' in tmp_key and 'adjusted' not in tmp_key:
        hdr_close = tmp_key
    elif 'adjusted close' in tmp_key:
        hdr_adjclose = tmp_key
    elif 'volume' in tmp_key:
        hdr_volume = tmp_key
    elif 'dividend' in tmp_key:
        hdr_dividend = tmp_key
    elif 'split' in tmp_key:
        hdr_split = tmp_key

print(hdr_open, hdr_high, hdr_low, hdr_close, hdr_adjclose, hdr_volume, hdr_dividend, hdr_split)

In [ ]:
list_data = []

bridge = {
    'open': hdr_open,
    'high': hdr_high,
    'low': hdr_low,
    'close': hdr_close,
    'adjclose': hdr_adjclose,
    'volume': hdr_volume,
    'div': hdr_dividend,
    'split': hdr_split,
}

for tmp_date in dict_data[hdr_ts]:
    
    if tmp_date != datetime.date.today().strftime('%Y-%m-%d'):
    
        row = {
            'symbol': tmp_symbol,
            'dateeod': tmp_date,
        }

        for tmp_conv_num in bridge:
            try:
                row[tmp_conv_num] = float(dict_data[hdr_ts][tmp_date][bridge[tmp_conv_num]])
            except:
                row[tmp_conv_num] = None


        list_data.append( row )
        #break

list_data

In [ ]:
df = pd.DataFrame(list_data)
df.head()